In [ ]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F # Import F for functional operations
from torch.utils.data import Dataset, DataLoader
from transformers import SwinConfig, SwinModel
import torch.optim.lr_scheduler as lr_scheduler # Import scheduler

import albumentations as A
from albumentations.pytorch import ToTensorV2

# --- Cài đặt tham số cố định ---
IMG_SIZE = 256
BATCH_SIZE = 8
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {DEVICE}")

# Thay đổi đường dẫn thư mục tùy theo máy của bạn
train_img_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\train\image'
train_mask_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\train\label'
val_img_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\val\image'
val_mask_dir = r'C:\Users\Admin\Documents\Python Project\DPL Crack detection\UDTIRI-Crack Detection\val\label'

# --- Thu thập đường dẫn tệp ảnh và mask ---
train_img_paths = sorted([os.path.join(train_img_dir, f) for f in os.listdir(train_img_dir)])
train_mask_paths = sorted([os.path.join(train_mask_dir, f) for f in os.listdir(train_mask_dir)])
val_img_paths = sorted([os.path.join(val_img_dir, f) for f in os.listdir(val_img_dir)])
val_mask_paths = sorted([os.path.join(val_mask_dir, f) for f in os.listdir(val_mask_dir)])

class CrackDetectionDataset(Dataset):
    def __init__(self, image_filenames, mask_filenames, augment=False):
        self.image_filenames = image_filenames
        self.mask_filenames = mask_filenames
        self.augment = augment

        if len(self.image_filenames) != len(self.mask_filenames):
            raise ValueError("Số lượng tệp ảnh và tệp mask không khớp.")

    def __len__(self):
        return len(self.image_filenames)

    # Hàm thực hiện resize ảnh và mask về kích thước mong muốn (ban đầu)
    def resize_image_and_mask(self, img, mask, target_size=IMG_SIZE):
        img = cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_AREA)
        mask = cv2.resize(mask, (target_size, target_size), interpolation=cv2.INTER_NEAREST)
        return img, mask

    # Hàm thực hiện cắt ngẫu nhiên cho ảnh và mask
    def random_crop_image_and_mask(self, img, mask, crop_size=IMG_SIZE, min_scale=0.7):
        h, w = img.shape[:2]
        # Tính toán kích thước crop ngẫu nhiên, không nhỏ hơn min_scale của IMG_SIZE
        # Đảm bảo kích thước cắt không lớn hơn kích thước hiện tại của ảnh
        current_min_dim = min(h, w)
        crop_h = crop_w = int(random.uniform(min_scale, 1.0) * crop_size)
        
        # Đảm bảo crop_h, crop_w không lớn hơn kích thước hiện tại của ảnh
        crop_h = min(crop_h, h)
        crop_w = min(crop_w, w)

        if h == crop_h and w == crop_w: # Không cần cắt nếu kích thước đã khớp
            return img, mask

        # Chọn điểm bắt đầu ngẫu nhiên
        start_x = random.randint(0, w - crop_w)
        start_y = random.randint(0, h - crop_h)

        cropped_img = img[start_y:start_y + crop_h, start_x:start_x + crop_w]
        cropped_mask = mask[start_y:start_y + crop_h, start_x:start_x + crop_w]
        
        # Resize lại về kích thước IMG_SIZE sau khi cắt
        cropped_img = cv2.resize(cropped_img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)
        cropped_mask = cv2.resize(cropped_mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)

        return cropped_img, cropped_mask

    def __getitem__(self, idx):
        img = cv2.imread(self.image_filenames[idx])
        if img is None:
            raise ValueError(f"Không thể đọc tệp ảnh: {self.image_filenames[idx]}")

        mask = cv2.imread(self.mask_filenames[idx], cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise ValueError(f"Không thể đọc tệp mask: {self.mask_filenames[idx]}")

        # Chuyển đổi ảnh sang RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Áp dụng Augmentation
        if self.augment:
            # Random Crop đầu tiên để có các phần khác nhau của ảnh
            # Đảm bảo ảnh đủ lớn để crop, nếu không, resize trước
            if img.shape[0] < IMG_SIZE or img.shape[1] < IMG_SIZE:
                img, mask = self.resize_image_and_mask(img, mask, target_size=IMG_SIZE)
            
            # Chỉ thực hiện crop nếu ảnh đủ lớn sau resize hoặc nếu ban đầu đã lớn
            # Với `min_scale=0.7`, chúng ta sẽ cắt một vùng có kích thước từ 0.7*IMG_SIZE đến IMG_SIZE
            img, mask = self.random_crop_image_and_mask(img, mask, crop_size=IMG_SIZE, min_scale=0.7)

            # Lật Ngang
            if random.random() < 0.5:
                img = cv2.flip(img, 1)
                mask = cv2.flip(mask, 1)
            # Lật Dọc
            if random.random() < 0.5:
                img = cv2.flip(img, 0)
                mask = cv2.flip(mask, 0)
        else:
            # Nếu không augment, chỉ resize về đúng kích thước IMG_SIZE
            img, mask = self.resize_image_and_mask(img, mask, target_size=IMG_SIZE)

        # Chuẩn hóa ảnh và mask về [0, 1]
        img = img.astype(np.float32) / 255.0
        mask = mask.astype(np.float32) / 255.0
        
        # Đảm bảo mask có chiều kênh (1, H, W)
        if mask.ndim == 2: # Nếu mask vẫn là (H, W)
            mask = np.expand_dims(mask, axis=0) # Thêm chiều kênh ở vị trí 0 -> (1, H, W)
        elif mask.ndim == 3 and mask.shape[0] != 1: # Nếu là (C, H, W) nhưng C không phải 1
            if mask.shape[0] == 3: # Nếu là 3 kênh (ví dụ, mask gốc là RGB)
                mask = mask[0:1, :, :] # Lấy kênh đầu tiên
            else:
                raise ValueError(f"Mask có hình dạng không mong muốn {mask.shape} tại chỉ số {idx} sau khi biến đổi. Expected channel dim 1.")
        
        # Chuyển đổi từ NumPy array sang PyTorch tensor
        img_tensor = torch.from_numpy(img).permute(2, 0, 1) # Ảnh: (H, W, C) -> (C, H, W)
        mask_tensor = torch.from_numpy(mask) # Mask đã ở (1, H, W)

        return img_tensor, mask_tensor

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        # Ưu tiên sử dụng nn.Upsample để tránh checkerboard artifacts
        self.upsample = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True), # Lên gấp đôi kích thước không gian
            nn.Conv2d(in_channels, out_channels, kernel_size=1)
        )

        self.conv_block = ConvBlock(out_channels + skip_channels, out_channels)

    def forward(self, x, skip_features=None):
        x = self.upsample(x) 
        
        if skip_features is not None:
            # Bước 2: Đảm bảo kích thước không gian của skip_features khớp với x.
            # Đây là điểm sửa lỗi: nội suy skip_features để nó khớp với kích thước đã được upsample của x.
            if x.shape[2:] != skip_features.shape[2:]:
                skip_features = F.interpolate(skip_features, size=x.shape[2:], mode='bilinear', align_corners=True)
            
            # Bước 3: Nối (concatenate) x và skip_features theo chiều kênh
            x = torch.cat([x, skip_features], dim=1)
        
        # Bước 4: Chạy qua khối convolution để xử lý các đặc trưng đã nối
        x = self.conv_block(x)
        return x
    
class SwinUNet(nn.Module):
    def __init__(self, input_channels=3, num_classes=1):
        super().__init__()
        self.IMG_SIZE = IMG_SIZE

        config = SwinConfig(image_size=self.IMG_SIZE, num_channels=input_channels,
                            patch_size=4, embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                            window_size=7, mlp_ratio=4., qkv_bias=True, hidden_dropout_prob=0.0,
                            attention_probs_dropout_prob=0.0, drop_path_rate=0.1,
                            hidden_act="gelu", use_absolute_embeddings=False,
                            patch_norm=True, initializer_range=0.02, layer_norm_eps=1e-05,
                            out_features=["stage1", "stage2", "stage3", "stage4"])
        self.swin = SwinModel(config)

        # Bottleneck: input từ hidden_state[3] (768 kênh), vì hidden_state[4] là sau LayerNorm và có thể không cần thiết
        self.bottleneck = ConvBlock(config.embed_dim * 8, config.embed_dim * 8) # (768, 768)

        # Decoder 4: in_channels từ bottleneck (768), skip từ hidden_state[3] (768 kênh), ra 384 kênh
        self.decoder4 = DecoderBlock(in_channels=config.embed_dim * 8, skip_channels=config.embed_dim * 8, out_channels=config.embed_dim * 4) # 768, 768, 384
        
        # Decoder 3: in_channels từ decoder4 (384), skip từ hidden_state[2] (384 kênh), ra 192 kênh
        self.decoder3 = DecoderBlock(in_channels=config.embed_dim * 4, skip_channels=config.embed_dim * 4, out_channels=config.embed_dim * 2) # 384, 384, 192
        
        # Decoder 2: in_channels từ decoder3 (192), upsample to 96, skip từ hidden_state[1] (192 kênh), ra 96 kênh
        self.decoder2 = DecoderBlock(in_channels=config.embed_dim * 2, skip_channels=config.embed_dim * 2, out_channels=config.embed_dim * 1) # 192, 192, 96
        
        # Decoder 1: in_channels từ decoder2 (96), upsample to 48, skip từ hidden_state[0] (96 kênh), ra 48 kênh
        self.decoder1 = DecoderBlock(in_channels=config.embed_dim * 1, skip_channels=config.embed_dim * 1, out_channels=config.embed_dim // 2) # 96, 96, 48
        
        self.final_upsample = DecoderBlock(in_channels=config.embed_dim // 2, skip_channels=0, out_channels=config.embed_dim // 4) # 48, 0, 24
        self.final_conv = nn.Conv2d(config.embed_dim // 4, num_classes, kernel_size=1)


    def forward(self, x):
        outputs = self.swin(pixel_values=x, output_hidden_states=True)
        encoder_features = []

        hs_skip_res_H4 = outputs.hidden_states[0] 
        b, n, c = hs_skip_res_H4.shape
        s = int(np.sqrt(n))
        encoder_features.append(hs_skip_res_H4.permute(0, 2, 1).reshape(b, c, s, s)) # index 0

        # Skip connection cho decoder2 (H/8): hidden_state[1]
        hs_skip_res_H8 = outputs.hidden_states[1] 
        b, n, c = hs_skip_res_H8.shape
        s = int(np.sqrt(n))
        encoder_features.append(hs_skip_res_H8.permute(0, 2, 1).reshape(b, c, s, s)) # index 1

        # Skip connection cho decoder3 (H/16): hidden_state[2]
        hs_skip_res_H16 = outputs.hidden_states[2] 
        b, n, c = hs_skip_res_H16.shape
        s = int(np.sqrt(n))
        encoder_features.append(hs_skip_res_H16.permute(0, 2, 1).reshape(b, c, s, s)) # index 2

        # Skip connection cho decoder4 (H/32): hidden_state[3]
        hs_skip_res_H32 = outputs.hidden_states[3] 
        b, n, c = hs_skip_res_H32.shape
        s = int(np.sqrt(n))
        encoder_features.append(hs_skip_res_H32.permute(0, 2, 1).reshape(b, c, s, s)) # index 3

        x_bottleneck = outputs.hidden_states[3] 
        b, n, c = x_bottleneck.shape
        s = int(np.sqrt(n))
        x_bottleneck = x_bottleneck.permute(0, 2, 1).reshape(b, c, s, s)
        
        x = self.bottleneck(x_bottleneck)

        # decoder4: x_in từ bottleneck (768), upsample to 384, skip từ encoder_features[3] (768 kênh)
        x = self.decoder4(x, encoder_features[3])
        # x_out của decoder4 là 384 kênh, 16x16

        # decoder3: x_in từ decoder4 (384), upsample to 192, skip từ encoder_features[2] (384 kênh)
        x = self.decoder3(x, encoder_features[2])
        # x_out của decoder3 là 192 kênh, 32x32

        # decoder2: x_in từ decoder3 (192), upsample to 96, skip từ encoder_features[1] (192 kênh)
        x = self.decoder2(x, encoder_features[1])
        # x_out của decoder2 là 96 kênh, 64x64
        
        # decoder1: x_in từ decoder2 (96), upsample to 48, skip từ encoder_features[0] (96 kênh)
        x = self.decoder1(x, encoder_features[0])
        
        x = self.final_upsample(x)
        outputs = self.final_conv(x)

        return outputs

class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, inputs, targets):

        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum() 
        dice = (2.*intersection + self.smooth)/(inputs.sum() + targets.sum() + self.smooth) 
        
        return 1 - dice

class CombinedLoss(nn.Module):
    def __init__(self, bce_weight=0.5, dice_weight=0.5, pos_weight=None, smooth=1e-6):
        super(CombinedLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.dice = DiceLoss(smooth=smooth)
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)

        sigmoid_inputs = torch.sigmoid(inputs)
        dice_loss = self.dice(sigmoid_inputs, targets)
        
        return self.bce_weight * bce_loss + self.dice_weight * dice_loss

def calculate_metrics(predicted_masks, true_masks, smooth=1e-6):
    
    intersection = (predicted_masks * true_masks).sum()
    union = (predicted_masks + true_masks).sum() - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    
    dice = (2. * intersection + smooth) / ((predicted_masks.sum() + true_masks.sum()) + smooth)
    f1_score = dice 

    return iou.item(), f1_score.item()

def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs, callbacks_config, start_epoch=0, best_val_loss_so_far=float('inf')):
    best_val_loss = best_val_loss_so_far 
    patience_counter = 0
    model_checkpoint_path = callbacks_config.get('checkpoint_path', 'swin_unet_best_pytorch.pth')
    
    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        running_iou = 0.0
        running_f1 = 0.0
        print(f"Epoch {epoch+1}/{num_epochs} Bắt đầu...")
        for batch_idx, (images, masks) in enumerate(train_loader):
            images = images.to(DEVICE)
            masks = masks.to(DEVICE)

            optimizer.zero_grad()

            outputs = model(images)

            # swin_outputs_for_debug = model.swin(pixel_values=images, output_hidden_states=True) # Dòng này không cần thiết trong quá trình huấn luyện thông thường

            loss = criterion(outputs, masks) # criterion sẽ xử lý sigmoid bên trong cho Dice loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            # Để tính metrics, cần áp dụng sigmoid cho outputs và sau đó ngưỡng
            predicted_masks = (torch.sigmoid(outputs) > 0.5).float()
            
            batch_iou, batch_f1 = calculate_metrics(predicted_masks, masks)
            running_iou += batch_iou * images.size(0)
            running_f1 += batch_f1 * images.size(0)

            if (batch_idx + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_iou = running_iou / len(train_loader.dataset)
        epoch_f1 = running_f1 / len(train_loader.dataset)
        print(f"Epoch {epoch+1} Kết thúc - Mất mát Huấn luyện: {epoch_loss:.4f}, IoU Huấn luyện: {epoch_iou:.4f}, F1-Score Huấn luyện: {epoch_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_iou = 0.0
        val_f1 = 0.0

        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(DEVICE)
                masks = masks.to(DEVICE)

                outputs = model(images) # outputs là logits
                loss = criterion(outputs, masks) # criterion sẽ xử lý sigmoid bên trong

                val_loss += loss.item() * images.size(0)

                # Để tính metrics, cần áp dụng sigmoid cho outputs và sau đó ngưỡng
                predicted_masks = (torch.sigmoid(outputs) > 0.5).float()
                
                batch_iou, batch_f1 = calculate_metrics(predicted_masks, masks)
                val_iou += batch_iou * images.size(0)
                val_f1 += batch_f1 * images.size(0)

        val_loss /= len(val_loader.dataset)
        val_iou /= len(val_loader.dataset)
        val_f1 /= len(val_loader.dataset)
        print(f"Mất mát Xác thực: {val_loss:.4f}, IoU Xác thực: {val_iou:.4f}, F1-Score Xác thực: {val_f1:.4f}")

        # CẬP NHẬT SCHEDULER DỰA TRÊN VAL_LOSS
        scheduler.step(val_loss) # Quan trọng: truyền val_loss vào scheduler

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            print(f"Mất mát xác thực tốt nhất được cập nhật: {best_val_loss:.4f}. Lưu mô hình và trạng thái...")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss,
                'scheduler_state_dict': scheduler.state_dict() # Lưu trạng thái scheduler
            }, model_checkpoint_path)
        else:
            patience_counter += 1
            print(f"Mất mát xác thực không cải thiện. Sự kiên nhẫn: {patience_counter}/{callbacks_config['patience']}")
            if patience_counter >= callbacks_config['patience']:
                print("Dừng sớm!")
                break

# --- Chuẩn bị dữ liệu ---
train_dataset = CrackDetectionDataset(train_img_paths, train_mask_paths, augment=True)
val_dataset = CrackDetectionDataset(val_img_paths, val_mask_paths, augment=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# --- Khởi tạo mô hình, optimizer, criterion ---
model = SwinUNet(input_channels=3, num_classes=1).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-3)

# Khởi tạo Scheduler (sau optimizer)
scheduler = lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',         # Giảm LR khi val_loss ngừng giảm
    factor=0.5,         # Hệ số giảm LR (ví dụ: LR mới = LR cũ * 0.5)
    patience=10,        # Số epoch chờ đợi trước khi giảm LR
    threshold=0.0001,   # Ngưỡng cải thiện tối thiểu
    threshold_mode='rel',
    cooldown=0,
    min_lr=1e-7,        # Tốc độ học tối thiểu
    verbose=True        # In thông báo khi LR thay đổi
)

my_pos_weight = 11
pos_weight_tensor = torch.tensor(my_pos_weight, dtype=torch.float).to(DEVICE)

criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.5, pos_weight=pos_weight_tensor)

print(model)

callbacks_config = {
    'patience': 30,
    'checkpoint_path': 'swin_unet_best_pytorchDiceloss.pth'
}

start_epoch = 0
best_val_loss_so_far = float('inf')
checkpoint_path = callbacks_config['checkpoint_path']

if os.path.exists(checkpoint_path):
    print(f"Phát hiện checkpoint tại {checkpoint_path}. Đang tải để tiếp tục huấn luyện...")
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # Tải trạng thái scheduler
    if 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    start_epoch = checkpoint['epoch'] + 1 
    best_val_loss_so_far = checkpoint['best_val_loss']
    
    print(f"Đã tải checkpoint từ Epoch {start_epoch-1}. Tiếp tục huấn luyện từ Epoch {start_epoch}.")
    print(f"Mất mát xác thực tốt nhất trước đó: {best_val_loss_so_far:.4f}")
else:
    print("Không tìm thấy checkpoint. Bắt đầu huấn luyện từ đầu (Epoch 0).")

print("\nBắt đầu huấn luyện mô hình Swin-Unet...")
train_model(model, train_loader, val_loader, optimizer, criterion, scheduler,
            num_epochs=10000, callbacks_config=callbacks_config,
            start_epoch=start_epoch, best_val_loss_so_far=best_val_loss_so_far)

print("\nQuá trình huấn luyện đã hoàn tất.")

c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Sử dụng thiết bị: cuda


c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_15064\3353620380.py:447: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use

SwinUNet(
  (swin): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96, bias=True)
                  (value): Linear(in_features=96, out_features=96, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOutput(
                  (dense):